In [6]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import brickschema
import datetime
from relaxation_graphs.fast_relaxation_graph import *
from evaluation.similarity_score import *
import uuid
import pickle
import time 
from rdflib import URIRef, Literal, BNode
from queries.mortar import queries as q1
%matplotlib inline

In [7]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 32}

matplotlib.rc('font', **font)

In [8]:
useful_queries = [
"""SELECT ?sensor WHERE {
    ?sensor rdf:type brick:Zone_Air_Temperature_Sensor .
    ?sensor brick:isPointOf ?equip 
}""",
"""SELECT ?sp WHERE {
    ?sp rdf:type brick:Zone_Air_Temperature_Setpoint .
    ?sp brick:isPointOf ?equip
}""",
"""SELECT ?sensor ?sp ?equip WHERE {
    ?sensor  rdf:type     brick:Air_Flow_Sensor .
    ?sp      rdf:type     brick:Air_Flow_Setpoint .
    ?sensor  brick:isPointOf ?equip .
    ?sp      brick:isPointOf ?equip
}""",
"""SELECT * WHERE {
    ?equip    rdf:type  brick:VAV .
    ?equip    brick:hasPoint  ?air_flow .
    ?air_flow rdf:type   brick:Supply_Air_Flow_Sensor
    }""",
"""SELECT ?vav WHERE {
    ?vav rdf:type brick:VAV
}""",
"""SELECT ?floor ?room WHERE {
    ?floor rdf:type brick:Floor .
    ?room rdf:type brick:Room .
    ?room brick:isPartOf+ ?floor .
}""",
    
"""SELECT * WHERE {
    ?oat_damper a brick:Outside_Damper .
    ?pos a brick:Damper_Position_Command .
    ?oat_damper brick:hasPoint ?pos .
    ?oat a brick:Outside_Air_Temperature_Sensor
}""",
    
"""SELECT * WHERE {
    ?equip   rdf:type  brick:VAV .
    ?equip   brick:isFedBy  ?ahu .
    ?ahu     brick:hasPoint  ?upstream_ta .
    ?equip   brick:hasPoint  ?dnstream_ta .
    ?upstream_ta  rdf:type brick:Supply_Air_Temperature_Sensor .
    ?dnstream_ta  rdf:type brick:Supply_Air_Temperature_Sensor .
    ?equip   brick:hasPoint  ?vlv .
    ?vlv     rdf:type   brick:Valve_Command
}"""
]

In [9]:
useful_buildings = [31, 35, 37]

files = os.listdir('Data - Brick models/')
brick_models = []

with open("brick_models.pkl", 'rb') as fp:
    brick_models = pickle.load(fp)
    

In [19]:
i=0
max_level=3
for q in useful_queries:
    count = 0
    t = 0
    st = time.time()
    G = get_fast_relaxed_graph(query=q, max_level=max_level)
    et = time.time()
    t += (et-st)*1000
    print(len(G))

8
8
26
17
5
28
31
421


In [11]:
i=0
for q in useful_queries:
    count = 0
    t = 0
    while count < 100:
        st = time.time()
        G = get_fast_relaxed_graph(query=q, max_level=max_level)
        et = time.time()
        t += (et-st)*1000
        count+=1
    print(round(t/100, 2))

0.49
0.42
0.8
0.74
0.19
0.62
0.95
49.42
